# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [1]:
import pandas as pd
from scipy.stats import shapiro
from scipy.stats import ttest_1samp
import numpy as np
import psycopg2
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_squared_error 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
test = pd.read_csv('flights_test.csv')

In [3]:
df = pd.read_csv('flights_merged.csv', index_col=0)

In [4]:
df = df.dropna(axis=0, subset=['crs_elapsed_time'])

In [5]:
df = df.sample(frac=0.1)

In [5]:
df

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,distance,passengers,capacity,total_gallons,dep_delay,nas_delay,security_delay,late_aircraft_delay,weather_delay,carrier_delay
0,2019-01-01,AA,AA_CODESHARE,AA,5606,OH,N575NN,5606,13577,MYR,...,157.0,1325.400000,157.0,0.000000e+00,2.76087,3.875,0.0,30.0,0.0,6.5
1,2019-01-01,AA,AA_CODESHARE,AA,5243,OH,N591NN,5243,13577,MYR,...,157.0,1325.400000,157.0,0.000000e+00,2.76087,3.875,0.0,30.0,0.0,6.5
2,2019-01-01,AA,AA_CODESHARE,AA,5249,OH,N578NN,5249,13577,MYR,...,157.0,1325.400000,157.0,0.000000e+00,2.76087,3.875,0.0,30.0,0.0,6.5
3,2019-01-01,AA,AA_CODESHARE,AA,5261,OH,N584NN,5261,13577,MYR,...,157.0,1325.400000,157.0,0.000000e+00,2.76087,3.875,0.0,30.0,0.0,6.5
4,2019-01-01,AA,AA_CODESHARE,AA,5358,OH,N577NN,5358,13577,MYR,...,157.0,1325.400000,157.0,0.000000e+00,2.76087,3.875,0.0,30.0,0.0,6.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195291,2018-01-04,F9,F9,F9,1445,F9,N919FR,1445,10397,ATL,...,874.0,1292.000000,196.0,2.353042e+07,74.00000,2.000,0.0,95.0,0.0,13.5
195292,2018-01-07,F9,F9,F9,1445,F9,N902FR,1445,10397,ATL,...,874.0,1292.000000,196.0,2.353042e+07,74.00000,2.000,0.0,95.0,0.0,13.5
195293,2018-01-02,F9,F9,F9,1111,F9,N313FR,1111,12266,IAH,...,1222.0,1324.333333,129.0,2.353042e+07,9.00000,20.000,0.0,0.0,7.0,0.0
195294,2018-01-04,F9,F9,F9,1111,F9,N211FR,1111,12266,IAH,...,1222.0,1324.333333,129.0,2.353042e+07,9.00000,20.000,0.0,0.0,7.0,0.0


In [6]:
y= df['arr_delay']

In [7]:
X = df

In [8]:
X = X.drop(columns=['arr_delay'])

In [9]:
X.fl_date = X.fl_date.apply(pd.to_datetime)

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [10]:
X = X[['origin_airport_id', 'dest_airport_id', 'distance', 'passengers', 'crs_elapsed_time','capacity', 'total_gallons', 'dep_delay',
       'nas_delay', 'security_delay', 'late_aircraft_delay',
       'weather_delay', 'carrier_delay']]
#dropped flight date :( 

In [11]:
x_label = X.columns

In [12]:
scaler = StandardScaler()

In [13]:
X = pd.DataFrame(scaler.fit_transform(X))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, train_size=0.8)


### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

In [215]:
#data_dmatrix = xgb.DMatrix(data=X,label=y, enable_categorical=True)

In [216]:
#objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1, max_depth = 7, alpha = 0.1,

In [15]:
lr = LinearRegression()

In [16]:
lr.fit(X_train, y_train) 

LinearRegression()

In [17]:
lr.score(X_train, y_train)

0.07173831801102315

In [18]:
lr.score(X_test, y_test)

0.07188307040310415

### Random Forest

RandomForestRegressor()

In [19]:
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 100)
 

In [20]:
# fit the regressor with x and y data
regressor.fit(X_train, y_train) 

RandomForestRegressor()

In [21]:
y_pred = regressor.predict(X_test)

In [22]:
print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 20.997776081386732
Mean Squared Error: 1031.3092705195854
Root Mean Squared Error: 32.11400427414161


In [23]:
feature_importance= pd.Series(regressor.feature_importances_, index= x_label).sort_values(ascending=False)
feature_importance

crs_elapsed_time       0.497925
dep_delay              0.176867
total_gallons          0.053538
distance               0.040849
nas_delay              0.036052
carrier_delay          0.033856
late_aircraft_delay    0.032412
capacity               0.028833
passengers             0.028475
origin_airport_id      0.027593
dest_airport_id        0.026465
weather_delay          0.014511
security_delay         0.002622
dtype: float64

In [25]:
params = {'n_estimators' : [100, 150, 200, 300], 'max_depth': [3, 10, 20, 30]}

In [26]:
grid = GridSearchCV(regressor, params)

In [27]:
grid_result = grid.fit(X_train, y_train)

In [28]:
print(grid_result.best_score_)
print( grid_result.best_params_['n_estimators'])
print( grid_result.best_params_['max_depth'])

0.11898671987936302
100
3


### XGBOOST

In [29]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 7, alpha = 0.1,)

In [ ]:
xg_reg.fit(X_train,y_train)


In [ ]:
preds = xg_reg.predict(X_test)

In [ ]:
xg_reg.score(X_train,y_train)

In [ ]:
xg_reg.score(X_test,y_test)

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.